# TP-2 Organización de Datos

# Oh La La... Data!

* **Hamma AALI CHTOUKI** --- *Padrón 106607 - Alumno regular de intercambio.*  
* **Aymeryc COUSAERT** ------  *Padrón 105464 - Alumno regular de intercambio.*
* **Mariana VINYOLAS**  ---------  *Alumna oyente.*

<!-- ![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
<a id='section2'></a>

<div style="border-bottom:3px solid #000;"> 
<div align="left"><h2>Step 2: Import de librerias </h2></div>
<div align="right">(return to <a href='#top1'><b>Top</b></a>)</div>
</div> -->



In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns


import re
import nltk.corpus
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import catboost as catb
import lightgbm as lgbm

import scikitplot as skplt
from scikitplot.metrics import plot_roc
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,classification_report, make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

C:\anaconda\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


<!-- ![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
<a id='section3'></a>

<div style="border-bottom:3px solid #000;"> 
<div align="left"><h2>Step 3: Feature engineering</h2></div>
<div align="right">(return to <a href='#top1'><b>Top</b></a>)</div>
</div> -->

In [5]:

df = train
df['text'] = df['text'].str.lower()   # pasamos a lowercase
df = df.drop(['id', 'location'], axis=1)

In [6]:
df.sample(8)

,keyword,text,target
6679,thunder,random wind gust just came through #gander. p...,1
2313,demolished,it was finally demolished in the spring of 201...,0
5607,razed,the latest: more homes razed by #northerncalif...,1
3439,exploded,@kopiteluke1892 its broken its fully exploded.,1
4851,mass%20murderer,#theatershooting defendant/mass murderer chose...,1
5157,obliterate,whereas jez will obliterate the national debt ...,1
1196,buildings%20burning,'i'm a gemini' *children screaming buildings b...,1
4298,hellfire,(also i dont think sewing thought a leather be...,0


#### Generamos nuevas features sobre el texto

In [7]:
# generamos una columna que indica la cantidad de links a  enlaces externos
df['link'] = df['text'].apply(lambda x: x.count('http'))

# generamos una columna que indica la cantidad de referencias a otras cuentas de twitter
df['contact'] = df['text'].apply(lambda x: x.count('@'))

# generamos una columna que indica la cantidad de hashtags
df['hashtag'] = df['text'].apply(lambda x: x.count('#'))

# generamos una columna que indica la cantidad de digitos
df['numerics'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

# calculamos la longitud del tweet andes de limpiar
df['length'] = df['text'].str.len()

In [8]:
# calculamos la cantidad de palabras antes de limpiar
def count_words(text):
    '''
    Funcion que toma un texto y devuelve la cantidad de palabras
    '''
    word_counts = len(text.split(' '))
    return word_counts

df['words'] = df['text'].apply(count_words)

In [9]:
# limpiamos el texto eliminando urls, cuentas, hashtags y numeros

def clean_str(string):
    string = re.sub(r'https?\://\S+', '', string)
    string = re.sub(r'http?\://\S+', '', string)
    string = re.sub(r'@\w*\s', '', string)
    string = re.sub(r'#\w*\s', '', string)
    string = re.sub(r'\d', '', string)
    return string

df['text_clean'] = df['text'].apply(lambda x: clean_str(str(x)))

In [10]:
df.sample(5)

,keyword,text,target,link,contact,hashtag,numerics,length,words,text_clean
5569,rainstorm,best believe all the wrong decisions are being...,0,0,0,0,0,128,23,best believe all the wrong decisions are being...
689,blazing,i'm the only weapons master here! let's go in ...,0,0,0,1,0,70,11,i'm the only weapons master here! let's go in ...
4244,hazardous,#foodscare #offers2go #nestleindia slips into ...,0,0,0,6,0,117,14,slips into loss after unsafe and hazardous for...
6751,tornado,dakota skye gets horny with some porn then get...,0,1,0,0,0,118,18,dakota skye gets horny with some porn then get...
5571,rainstorm,waking up sick with a rainstorm outside would ...,0,0,0,0,0,136,26,waking up sick with a rainstorm outside would ...


<!-- ![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
<a id='section4'></a>

<div style="border-bottom:3px solid #000;"> 
<div align="left"><h2>Step 4: NLP</h2></div>
<div align="right">(return to <a href='#top1'><b>Top</b></a>)</div>
</div> -->

In [11]:
# eliminamos stopwords

stop = stopwords.words('english')

df['text_clean'] = df['text_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [12]:
# eliminamos signos de puntuacion y caracteres especiales
df['text_clean'] = df['text_clean'].str.replace('[^\w\s]','')

In [13]:
df['word_tokenize'] = df['text_clean'].apply(lambda x: word_tokenize(x))

In [14]:
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

In [15]:
# unificamos palabras que poseen la misma raiz aplicando la funcion word_lematizer
df['word_lemmatizer'] = df['word_tokenize'].apply(lambda x: word_lemmatizer(x))

In [16]:
# unificamos la lista de tokens para poder analizar el texto limpio
df['text_clean'] = df['word_lemmatizer'].str.join(' ')

In [17]:
# calculamos nuevamente la longuitud, pero ahora del texto limpio                       
df['length-clean'] = df['text_clean'].str.len()

In [18]:
df['words_clean'] = df['text_clean'].apply(count_words)

In [19]:
pd.set_option('display.max_colwidth', 150)  

In [20]:
df[['text', 'text_clean']].sample(10)

,text,text_clean
5800,the last time a high profile name was due to be signing for #nffc the city was rioting! wesley verhoek now a household name! #don'tpanic,last time high profile name due signing city rioting wesley verhoek household name dontpanic
3767,this nigga cyhi diss was what meek was suppose to do. this shit actually fire,nigga cyhi dis meek suppose do shit actually fire
761,just realized my dude @_onlyftf was on that 'what are those' way before it blew up @ the tusky ?? game @robsimss @cantmisskid,realized dude what those way blew tusky game cantmisskid
5066,this is the natural and unavoidable consequence of socialism everywhere it has been tried.\nhttp://t.co/bbdpnj8xsx a,natural unavoidable consequence socialism everywhere tried
1009,@mzgraciebaby for the record im jumpin out the window early... i got @officialrealrap body bagging luck.. lol save the file,record im jumpin window early got body bagging luck lol save file
705,@asukager magical bag of blazing,magical bag blazing
7263,it was a whirlwind love affair that began over back fat and grew into much more.,whirlwind love affair began back fat grew much more
1044,bitches be takin pics with bags bigger than they whole body ??????,bitch takin pic bag bigger whole body
5105,bombing #iran would result in a never-ending game of #nuclear whack-a-mole. here's why: http://t.co/6exs23muy3 http://t.co/l9idherotj,bombing would result neverending game whackamole here why
5004,@cochisecollege for the people who died in human experiments by unit 731 of japanese military http://t.co/vvplfqv58p http://t.co/ldx9ukngsk,people died human experiment unit japanese military


In [21]:
df['keyword'] = df.keyword.str.replace('%20', '_')
df['keyword'] = df.keyword.str.lower()

In [22]:
df.sample(3)

,keyword,text,target,link,contact,hashtag,numerics,length,words,text_clean,word_tokenize,word_lemmatizer,length-clean,words_clean
2390,derail,dozens dead as two trains derail over river in india http://t.co/zkkn6mse1n http://t.co/fzhjf8bxld,1,2,0,0,0,98,12,dozen dead two train derail river india,"[dozens, dead, two, trains, derail, river, india]","[dozen, dead, two, train, derail, river, india]",39,7
3646,fatalities,sharing to help our cousin's family http://t.co/lsjowgyvqh,0,1,0,0,0,58,7,sharing help cousin family,"[sharing, help, cousins, family]","[sharing, help, cousin, family]",26,4
2916,drown,drown by bring me the horizon (at information resources centre (utp)) ûó https://t.co/7vsqqsvgni,0,1,0,0,0,97,13,drown bring horizon at information resource centre utp ûó,"[drown, bring, horizon, at, information, resources, centre, utp, ûó]","[drown, bring, horizon, at, information, resource, centre, utp, ûó]",57,9


## TF- IDF

In [23]:
# aplicamos TF-IDF seteando un maximo de 1500 palabras
tfidf = TfidfVectorizer(max_features=1500, lowercase=True, analyzer='word', stop_words= 'english',ngram_range=(1,1))


train_vect = tfidf.fit_transform(df['text_clean'])                                             

# lo pasamos a dataframe
df_tf_idf = pd.DataFrame(data = train_vect.todense(), columns = tfidf.get_feature_names())

In [24]:
df_tf_idf.shape

(7613, 1500)

In [25]:
df_tf_idf['target'].value_counts()

0.000000    7602
0.750117       3
0.317217       1
0.769954       1
0.758087       1
0.322396       1
0.601328       1
0.513594       1
0.493300       1
0.437970       1
Name: target, dtype: int64

In [26]:
df_tf_idf.head()

,aba,abandoned,abc,ablaze,able,absolutely,access,accident,according,account,...,york,young,youth,youtube,yr,zone,û_,ûïwhen,ûò,ûó
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
df_tf_idf['target'].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: target, dtype: float64

In [28]:
# como existe la columna target en el analisis tf-idf, la modifico para hacer el concat con 'target' y que no se duplique
df_tf_idf['targ'] = df_tf_idf['target']
df_tf_idf = df_tf_idf.drop(['target'], axis=1)

Para no perder tantos registros, vamos a dejar de lado por ahora la columna ``keyword`` 

In [29]:
df['target'].value_counts()  # chequeamos si los labels estan balanceados

0    4342
1    3271
Name: target, dtype: int64

In [30]:
# eliminamos col que contienen texto y conservamos unicamente las numericas
df_num = df.drop([ 'keyword', 'text', 'text_clean', 'word_tokenize', 'word_lemmatizer'], axis=1)

In [31]:
df_train = pd.concat([df_num, df_tf_idf], axis=1)  # revisar porque aparecen dos columnas llamadas target

In [32]:
df_train.shape

(7613, 1509)

## Train Test Split

In [33]:
# separamos el target del resto de los features

y = df_train.target    
X = df_train.drop('target', axis=1)

In [34]:
X.shape, y.shape

((7613, 1508), (7613,))

In [35]:
# hacemos division entre train y test para cross validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)      

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5329, 1508), (2284, 1508), (5329,), (2284,))

In [36]:
# estandarizamos las features
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test) 

In [37]:
# definimos una 
cv = StratifiedKFold(n_splits=5, random_state=27, shuffle=True)

## CatBoost

In [38]:
model_cat = catb.CatBoostClassifier()

In [39]:
model_cat.fit(X_train, y_train)

Learning rate set to 0.026901
0:	learn: 0.6899257	total: 416ms	remaining: 6m 55s
1:	learn: 0.6872060	total: 635ms	remaining: 5m 17s
2:	learn: 0.6841018	total: 838ms	remaining: 4m 38s
3:	learn: 0.6811722	total: 1.07s	remaining: 4m 27s
4:	learn: 0.6786617	total: 1.3s	remaining: 4m 18s
5:	learn: 0.6754396	total: 1.54s	remaining: 4m 15s
6:	learn: 0.6731550	total: 1.76s	remaining: 4m 10s
7:	learn: 0.6708433	total: 1.99s	remaining: 4m 6s
8:	learn: 0.6680880	total: 2.23s	remaining: 4m 6s
9:	learn: 0.6659282	total: 2.44s	remaining: 4m 2s
10:	learn: 0.6634854	total: 2.71s	remaining: 4m 3s
11:	learn: 0.6614379	total: 2.93s	remaining: 4m 1s
12:	learn: 0.6600594	total: 3.14s	remaining: 3m 58s
13:	learn: 0.6585267	total: 3.4s	remaining: 3m 59s
14:	learn: 0.6565543	total: 3.6s	remaining: 3m 56s
15:	learn: 0.6548567	total: 3.8s	remaining: 3m 53s
16:	learn: 0.6534525	total: 4.01s	remaining: 3m 51s
17:	learn: 0.6521807	total: 4.24s	remaining: 3m 51s
18:	learn: 0.6505802	total: 4.48s	remaining: 3m 51s
1

157:	learn: 0.5637147	total: 35.3s	remaining: 3m 7s
158:	learn: 0.5634248	total: 35.5s	remaining: 3m 7s
159:	learn: 0.5630126	total: 35.7s	remaining: 3m 7s
160:	learn: 0.5626819	total: 36s	remaining: 3m 7s
161:	learn: 0.5622270	total: 36.3s	remaining: 3m 7s
162:	learn: 0.5619617	total: 36.5s	remaining: 3m 7s
163:	learn: 0.5616140	total: 36.7s	remaining: 3m 6s
164:	learn: 0.5612606	total: 36.9s	remaining: 3m 6s
165:	learn: 0.5609089	total: 37.1s	remaining: 3m 6s
166:	learn: 0.5606453	total: 37.4s	remaining: 3m 6s
167:	learn: 0.5604836	total: 37.6s	remaining: 3m 6s
168:	learn: 0.5601258	total: 37.8s	remaining: 3m 5s
169:	learn: 0.5598166	total: 38s	remaining: 3m 5s
170:	learn: 0.5596524	total: 38.2s	remaining: 3m 5s
171:	learn: 0.5594077	total: 38.4s	remaining: 3m 4s
172:	learn: 0.5590586	total: 38.6s	remaining: 3m 4s
173:	learn: 0.5588863	total: 38.9s	remaining: 3m 4s
174:	learn: 0.5584181	total: 39.1s	remaining: 3m 4s
175:	learn: 0.5580866	total: 39.3s	remaining: 3m 3s
176:	learn: 0.55

314:	learn: 0.5229250	total: 1m 9s	remaining: 2m 30s
315:	learn: 0.5225921	total: 1m 9s	remaining: 2m 30s
316:	learn: 0.5223429	total: 1m 9s	remaining: 2m 30s
317:	learn: 0.5219020	total: 1m 9s	remaining: 2m 29s
318:	learn: 0.5215881	total: 1m 10s	remaining: 2m 29s
319:	learn: 0.5212481	total: 1m 10s	remaining: 2m 29s
320:	learn: 0.5211824	total: 1m 10s	remaining: 2m 29s
321:	learn: 0.5209798	total: 1m 10s	remaining: 2m 29s
322:	learn: 0.5207231	total: 1m 11s	remaining: 2m 28s
323:	learn: 0.5204641	total: 1m 11s	remaining: 2m 28s
324:	learn: 0.5201908	total: 1m 11s	remaining: 2m 28s
325:	learn: 0.5199878	total: 1m 11s	remaining: 2m 28s
326:	learn: 0.5197865	total: 1m 11s	remaining: 2m 28s
327:	learn: 0.5194913	total: 1m 12s	remaining: 2m 27s
328:	learn: 0.5192096	total: 1m 12s	remaining: 2m 27s
329:	learn: 0.5189740	total: 1m 12s	remaining: 2m 27s
330:	learn: 0.5186248	total: 1m 12s	remaining: 2m 26s
331:	learn: 0.5182998	total: 1m 12s	remaining: 2m 26s
332:	learn: 0.5180113	total: 1m 

467:	learn: 0.4863558	total: 1m 42s	remaining: 1m 56s
468:	learn: 0.4861810	total: 1m 42s	remaining: 1m 56s
469:	learn: 0.4860324	total: 1m 42s	remaining: 1m 55s
470:	learn: 0.4857838	total: 1m 42s	remaining: 1m 55s
471:	learn: 0.4856540	total: 1m 43s	remaining: 1m 55s
472:	learn: 0.4852102	total: 1m 43s	remaining: 1m 55s
473:	learn: 0.4850102	total: 1m 43s	remaining: 1m 54s
474:	learn: 0.4847628	total: 1m 43s	remaining: 1m 54s
475:	learn: 0.4845451	total: 1m 43s	remaining: 1m 54s
476:	learn: 0.4842750	total: 1m 44s	remaining: 1m 54s
477:	learn: 0.4842291	total: 1m 44s	remaining: 1m 53s
478:	learn: 0.4840604	total: 1m 44s	remaining: 1m 53s
479:	learn: 0.4838135	total: 1m 44s	remaining: 1m 53s
480:	learn: 0.4835668	total: 1m 44s	remaining: 1m 53s
481:	learn: 0.4834112	total: 1m 45s	remaining: 1m 52s
482:	learn: 0.4831676	total: 1m 45s	remaining: 1m 52s
483:	learn: 0.4828707	total: 1m 45s	remaining: 1m 52s
484:	learn: 0.4826873	total: 1m 45s	remaining: 1m 52s
485:	learn: 0.4824513	total:

620:	learn: 0.4599363	total: 2m 13s	remaining: 1m 21s
621:	learn: 0.4598375	total: 2m 14s	remaining: 1m 21s
622:	learn: 0.4597712	total: 2m 14s	remaining: 1m 21s
623:	learn: 0.4596253	total: 2m 14s	remaining: 1m 21s
624:	learn: 0.4594679	total: 2m 14s	remaining: 1m 20s
625:	learn: 0.4594003	total: 2m 14s	remaining: 1m 20s
626:	learn: 0.4591689	total: 2m 15s	remaining: 1m 20s
627:	learn: 0.4590622	total: 2m 15s	remaining: 1m 20s
628:	learn: 0.4589348	total: 2m 15s	remaining: 1m 19s
629:	learn: 0.4588889	total: 2m 15s	remaining: 1m 19s
630:	learn: 0.4587786	total: 2m 15s	remaining: 1m 19s
631:	learn: 0.4586075	total: 2m 16s	remaining: 1m 19s
632:	learn: 0.4584688	total: 2m 16s	remaining: 1m 19s
633:	learn: 0.4583927	total: 2m 16s	remaining: 1m 18s
634:	learn: 0.4583301	total: 2m 16s	remaining: 1m 18s
635:	learn: 0.4582527	total: 2m 16s	remaining: 1m 18s
636:	learn: 0.4581210	total: 2m 17s	remaining: 1m 18s
637:	learn: 0.4580555	total: 2m 17s	remaining: 1m 17s
638:	learn: 0.4579649	total:

776:	learn: 0.4407672	total: 2m 46s	remaining: 47.8s
777:	learn: 0.4406240	total: 2m 46s	remaining: 47.6s
778:	learn: 0.4405594	total: 2m 47s	remaining: 47.4s
779:	learn: 0.4404324	total: 2m 47s	remaining: 47.2s
780:	learn: 0.4403490	total: 2m 47s	remaining: 46.9s
781:	learn: 0.4402893	total: 2m 47s	remaining: 46.7s
782:	learn: 0.4401323	total: 2m 47s	remaining: 46.5s
783:	learn: 0.4399491	total: 2m 47s	remaining: 46.3s
784:	learn: 0.4398062	total: 2m 48s	remaining: 46.1s
785:	learn: 0.4397608	total: 2m 48s	remaining: 45.8s
786:	learn: 0.4396737	total: 2m 48s	remaining: 45.6s
787:	learn: 0.4395581	total: 2m 48s	remaining: 45.4s
788:	learn: 0.4393645	total: 2m 48s	remaining: 45.2s
789:	learn: 0.4391889	total: 2m 49s	remaining: 45s
790:	learn: 0.4391030	total: 2m 49s	remaining: 44.8s
791:	learn: 0.4390655	total: 2m 49s	remaining: 44.5s
792:	learn: 0.4389572	total: 2m 49s	remaining: 44.3s
793:	learn: 0.4389393	total: 2m 49s	remaining: 44.1s
794:	learn: 0.4388879	total: 2m 50s	remaining: 4

933:	learn: 0.4252069	total: 3m 18s	remaining: 14.1s
934:	learn: 0.4251004	total: 3m 19s	remaining: 13.8s
935:	learn: 0.4250315	total: 3m 19s	remaining: 13.6s
936:	learn: 0.4249052	total: 3m 19s	remaining: 13.4s
937:	learn: 0.4248366	total: 3m 19s	remaining: 13.2s
938:	learn: 0.4248190	total: 3m 19s	remaining: 13s
939:	learn: 0.4247651	total: 3m 20s	remaining: 12.8s
940:	learn: 0.4247135	total: 3m 20s	remaining: 12.6s
941:	learn: 0.4246541	total: 3m 20s	remaining: 12.3s
942:	learn: 0.4245802	total: 3m 20s	remaining: 12.1s
943:	learn: 0.4244743	total: 3m 20s	remaining: 11.9s
944:	learn: 0.4243754	total: 3m 21s	remaining: 11.7s
945:	learn: 0.4243079	total: 3m 21s	remaining: 11.5s
946:	learn: 0.4240982	total: 3m 21s	remaining: 11.3s
947:	learn: 0.4240185	total: 3m 21s	remaining: 11.1s
948:	learn: 0.4239019	total: 3m 21s	remaining: 10.9s
949:	learn: 0.4238553	total: 3m 22s	remaining: 10.6s
950:	learn: 0.4237591	total: 3m 22s	remaining: 10.4s
951:	learn: 0.4236964	total: 3m 22s	remaining: 1

In [40]:
y_pred_cat = model_cat.predict(X_test)

<!-- ![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
<a id='section6'></a>

<div style="border-bottom:3px solid #000;"> 
<div align="left"><h2>Step 6: Scoring</h2></div>
<div align="right">(return to <a href='#top1'><b>Top</b></a>)</div>
</div> -->

In [41]:
model_cat.score(X_train, y_train), model_cat.score(X_test,y_test)

(0.8427472321261025, 0.7631348511383538)

In [42]:
0.7618213660245184 # sin estandarizar los datos

0.7618213660245184

In [43]:
0.7631348511383538  # con StandardScaler

0.7631348511383538

## GridSearch

In [ ]:
clf = catb.CatBoostClassifier()
params = {'iterations': [500],
          'depth': [4, 5, 6],
          'loss_function': ['Logloss', 'CrossEntropy'],
          'l2_leaf_reg': np.logspace(-20, -19, 3),
          'leaf_estimation_iterations': [10],
#           'eval_metric': ['Accuracy'],
#           'use_best_model': ['True'],
          'logging_level':['Silent'],
          'random_seed': [42]
         }
scorer = make_scorer(accuracy_score)
clf_grid = GridSearchCV(estimator=clf, param_grid=params, scoring=scorer, cv=cv)

In [ ]:
# clf_grid.fit(X_train, y_train)

In [ ]:
best_param = clf_grid.best_params_
best_param

In [ ]:
model = catb.CatBoostClassifier(iterations=1000,
                           loss_function=best_param['loss_function'],
                           depth=best_param['depth'],
                           l2_leaf_reg=best_param['l2_leaf_reg'],
                           eval_metric='Accuracy',
                           leaf_estimation_iterations=10,
#                            use_best_model=True,
                           logging_level='Silent',
                           random_seed=42
                          )

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
skplt.metrics.plot_confusion_matrix(y_test, y_pred, normalize=True, figsize=(8,8));